In [0]:
# Databricks notebook source
from pyspark.sql import SparkSession, functions as f

#Reading Hospital A departments data 
df_hos_1=spark.read.parquet("/mnt/bronze/Hospital-1/departments")

#Reading Hospital B departments data 
df_hos_2=spark.read.parquet("/mnt/bronze/Hospital-2/departments")

#union two departments dataframes
df_merged = df_hos_1.unionByName(df_hos_2)

# Create the dept_id column and rename deptid to src_dept_id
df_merged = df_merged.withColumn("SRC_Dept_id", f.col("deptid")) \
                     .withColumn("Dept_id", f.concat(f.col("deptid"),f.lit('-'), f.col("datasource"))) \
                     .drop("deptid")

In [0]:
df_merged.createOrReplaceTempView("departments")

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS healthcarerevenuecyclemanagement_databricks.silver

In [0]:
%sql
CREATE TABLE IF NOT EXISTS healthcarerevenuecyclemanagement_databricks.silver.departments(
  DeptID string,
  SRC_Dept_id string,
  Name string,
  datasource string,
  is_quarantined BOOLEAN
)
USING DELTA;

In [0]:
%sql
TRUNCATE TABLE silver.departments;

In [0]:
%sql
INSERT INTO healthcarerevenuecyclemanagement_databricks.silver.departments
SELECT
Dept_id,
SRC_Dept_id,
Name,
datasource,
CASE
  WHEN SRC_Dept_id IS NULL OR Name IS NULL THEN TRUE
  ELSE FALSE
END as is_quarantined
FROM departments;